## Libraries

In [31]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.decomposition import PCA   
import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics.classification import log_loss
from sklearn.utils import shuffle

## Load files

In [32]:
train_features = pd.read_csv("~/Documents/R/Kaggle/lish_moa/train_features.csv")
train_targets_scored = pd.read_csv("~/Documents/R/Kaggle/lish_moa/train_targets_scored.csv")
train_targets_nonscored = pd.read_csv("~/Documents/R/Kaggle/lish_moa/train_targets_nonscored.csv")

In [33]:
# join train_features and non_scored_forTrain
non_scored = pd.merge(train_targets_nonscored,train_features,on="sig_id", how="inner")   
score ={"score":non_scored.iloc[:,1:403].sum(axis=1)}
score=pd.DataFrame(data = score)
non_scored = pd.concat([non_scored, score], axis=1)
non_scored = non_scored[non_scored["score"]>0]
non_scored_forTrain = pd.concat([non_scored.iloc[:,0], 
                                 non_scored.iloc[:,403:1278]], axis=1)
train_features = pd.concat([train_features, non_scored_forTrain])
train_features_labels = pd.merge(train_features, train_targets_nonscored,
                                 how="left", on="sig_id")
train_features_labels = pd.merge(train_features, train_targets_scored, 
                                 how="left", on="sig_id")
# Features
train_features = train_features_labels.iloc[:, 1:876]
train_features.shape
# Labels
train_labels = train_features_labels.iloc[:, 876:1082]
train_labels.shape

(28404, 206)

<bound method _RepeatedSplits.split of RepeatedKFold(n_repeats=4, n_splits=5, random_state=123)>

In [34]:
def modelPca(n_components=80, n_estimators=50):        
    # PCA
    #import pdb; pdb.set_trace()
    embedding = PCA(n_components=n_components)
    train_features_pca = train_features.iloc[:, 3:]
    train_pca = embedding.fit_transform(train_features_pca)
    train_pca.shape
    train_features["cp_time"] = train_features["cp_time"].astype(str)
    dummies_train = train_features.iloc[:, 0:3]
    dummies_train = pd.get_dummies(data=dummies_train, drop_first=True)
    train_features_pca_dummies = pd.concat([pd.DataFrame(data=train_pca),
                                            dummies_train], axis=1)
    test_size = 0.85
    X_train, X_test, y_train, y_test = train_test_split(
        train_features_pca_dummies.to_numpy(),
        train_labels.to_numpy(),
        test_size=test_size,
        random_state=42)
    # random forest multioutput
    xgb= XGBClassifier(n_estimators = n_estimators)
    global classifier
    classifier = MultiOutputClassifier(xgb)
    classifier.fit(X_train, y_train)
    val_preds = classifier.predict_proba(X_test)
    val_preds = np.array(val_preds)[:,:,1].T
    global loss
    loss = log_loss(np.ravel(y_test),np.ravel(val_preds))
    print("loss :",format(round(loss,5)))


In [37]:
n_components=80

embedding = PCA(n_components=n_components)
train_features_pca = train_features.iloc[:, 3:]
train_pca = embedding.fit_transform(train_features_pca)
train_pca.shape
train_features["cp_time"] = train_features["cp_time"].astype(str)
dummies_train = train_features.iloc[:, 0:3]
dummies_train = pd.get_dummies(data=dummies_train, drop_first=True)
train_features_pca_dummies = pd.concat([pd.DataFrame(data=train_pca),
                                            dummies_train], axis=1)
#test_size = 0.85
X=train_features_pca_dummies.to_numpy()
y=train_labels.to_numpy()
#X_train, X_test, y_train, y_test = train_test_split(
#train_features_pca_dummies.to_numpy(),
#train_labels.to_numpy(),
#test_size=test_size,
#random_state=42)

In [43]:
print(X.shape)
print(y.shape)

(28404, 84)
(28404, 206)


In [47]:
n_estimators=50
xgb= XGBClassifier(n_estimators = n_estimators, verbose=True)
#global classifier
classifier = MultiOutputClassifier(xgb)
    #classifier.fit(X_train, y_train)
classifier

MultiOutputClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None, gamma=None,
                                              gpu_id=None,
                                              importance_type='gain',
                                              interaction_constraints=None,
                                              learning_rate=None,
                                              max_delta_step=None,
                                              max_depth=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=50, n_jobs=None,
                          

In [ ]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_validate
from sklearn import metrics
#from sklearn.metrics import log_loss
random_state = 123
rkf = RepeatedKFold(n_repeats=4, n_splits=5, random_state=random_state)
cv_model = cross_val_score(classifier, X,y,cv=rkf, scoring='f1_macro',verbose=True)
sorted(cv_model.keys())

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[13:02:55] WARNING: ../src/learner.cc:516: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:02:58] WARNING: ../src/learner.cc:516: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:03:02] WARNING: ../src/learner.cc:516: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:03:06] WARNING:

[13:05:13] WARNING: ../src/learner.cc:516: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:05:16] WARNING: ../src/learner.cc:516: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:05:21] WARNING: ../src/learner.cc:516: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:05:27] WARNING:

In [ ]:
modelPca()

In [5]:
xgb_loss_df = pd.DataFrame()
n_components = [2,10,20,40,60,80,100]
n_estimators = [20,60,100,200,500]

for c in range(0,len(n_components),1):
    for e in range(0, len(n_estimators),1):
        
        print(c, " # n_components: ", n_components[c])
        print(e, " # n_estimators: ", n_estimators[e])
        
        modelPca(n_components = n_components[c], n_estimators = n_estimators[e])
        kouvas = pd.DataFrame(data = {'n_components':n_components[c],
                                      'n_estimators':n_estimators[e],
                                      'loss':round(loss,5)},index=[0]
                              )
        
        xgb_loss_df = pd.concat([xgb_loss_df, kouvas])
        xgb_loss_df.to_csv("xgb_loss_df.csv")


plt.plot(xgb_loss_df['n_estimators'], xgb_loss_df['loss'])
plt.show()
plt.plot(xgb_loss_df['n_components'], xgb_loss_df['loss'])

xgb_loss_df

0  # n_components:  2
0  # n_estimators:  20


NameError: name 'modelPca' is not defined